In [3]:
# NOTE:
#   이 스크립트는 example.py Colab 워크플로우를 바탕으로
#   GOOD/BAD/BLUEHOLE 3개 클래스로 확장한 일회성 학습 파이프라인 예시입니다.
#   Colab(.ipynb) 문법을 일부 유지하며, 제공되지 않은 경로/라벨/폴더는 공란으로 두었습니다.

print("단계 1: 환경 설정 시작")

# ------------------------------------------------------------
# 단계 1: Colab 환경 세팅 및 의존성 설치
# ------------------------------------------------------------

from google.colab import drive  # type: ignore
import os
import shutil
import json
from pathlib import Path
from typing import Dict, Iterable, Optional

from PIL import Image  # type: ignore
import glob

from pathlib import Path

try:
    drive.mount('/content/drive')
    print("Google Drive 마운트 성공")
except Exception as e:
    print(f"Google Drive 마운트 오류: {e}")
    print("이미 마운트되었을 수도 있습니다.")

print("Ultralytics YOLO 설치 진행")

!pip install ultralytics

from ultralytics import YOLO

print("단계 1: 환경 설정 완료")


# ------------------------------------------------------------
# 단계 2: 데이터 경로 및 카테고리 정의
# ------------------------------------------------------------

print("\n단계 2: 데이터 경로 및 카테고리 정의")

base_drive_path = Path("/content/drive/MyDrive/Hackathon")

# 학습 데이터 (양품/불량/블루홀)
train_good_img_path = os.path.join(base_drive_path, "Training/source/TS_용접_용접양품_조인트")
train_good_lbl_path = os.path.join(base_drive_path, "Training/label/TL_용접_용접양품_조인트")
train_bad_img_path = os.path.join(base_drive_path, "Training/source/TS_용접_용접불량_조인트")
train_bad_lbl_path = os.path.join(base_drive_path, "Training/label/TL_용접_용접불량_조인트")
train_blue_hole_img_path = os.path.join(base_drive_path, "Training/source/TS_용접_용접블루홀_조인트")
train_blue_hole_lbl_path = os.path.join(base_drive_path, "Training/label/TL_용접_용접블루홀_조인트")

# 검증 데이터 (양품/불량/블루홀)
val_good_img_path = os.path.join(base_drive_path, "Validation/source/VS_용접_용접양품_조인트")
val_good_lbl_path = os.path.join(base_drive_path, "Validation/label/VL_용접_용접양품_조인트")
val_bad_img_path = os.path.join(base_drive_path, "Validation/source/VS_용접_용접불량_조인트")
val_bad_lbl_path = os.path.join(base_drive_path, "Validation/label/VL_용접_용접불량_조인트")
val_blue_hole_img_path = os.path.join(base_drive_path, "Validation/source/VS_용접_용접블루홀_조인트")
val_blue_hole_lbl_path = os.path.join(base_drive_path, "Validation/label/VL_용접_용접블루홀_조인트")

# 로컬(Colab) 처리용 데이터셋 경로
dataset_base_path = "/content/weld_dataset_multiclass"
for subdir in ("images/train", "images/val", "labels/train", "labels/val"):
    os.makedirs(os.path.join(dataset_base_path, subdir), exist_ok=True)

# 클래스 정의
CLASS_ID_MAP: Dict[str, int] = {"good_weld": 0, "bad_weld": 1, "bluehole_weld": 2}

GOOD_CATEGORIES: Iterable[int] = {1101}
BAD_CATEGORIES: Iterable[int] = {1102}
BLUEHOLE_CATEGORIES: Iterable[int] = {1202}

print("단계 2: 카테고리 및 경로 설정 완료")


# ------------------------------------------------------------
# 단계 3: COCO -> YOLO 변환 유틸 함수
# ------------------------------------------------------------

def normalize_coco_bbox(img_width: float, img_height: float, bbox):
    x_min, y_min, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_min + w / 2.0
    y_center = y_min + h / 2.0

    x_center_norm = x_center * dw
    y_center_norm = y_center * dh
    width_norm = w * dw
    height_norm = h * dh

    return x_center_norm, y_center_norm, width_norm, height_norm


def resolve_class_id(category_id: int) -> Optional[int]:
    if category_id in GOOD_CATEGORIES:
        return CLASS_ID_MAP["good_weld"]
    if category_id in BAD_CATEGORIES:
        return CLASS_ID_MAP["bad_weld"]
    if category_id in BLUEHOLE_CATEGORIES:
        return CLASS_ID_MAP["bluehole_weld"]
    return None


def convert_data(img_src_dir: str, lbl_src_dir: str, img_dest_dir: str, lbl_dest_dir: str, skip_sampling: bool = False):
    if not img_src_dir or not lbl_src_dir:
        print("공란 경로가 포함되어 변환을 건너뜁니다.")
        return

    json_files = glob.glob(os.path.join(lbl_src_dir, "*.json"))
    total_files = len(json_files)

    if total_files == 0:
        print(f"경고: {lbl_src_dir}에서 JSON 파일을 찾지 못했습니다.")
        return

    if skip_sampling:
        files_to_process = json_files
    else:
        num_to_process = max(1, int(total_files * 0.1))
        files_to_process = json_files[:num_to_process]

    print(f"{lbl_src_dir} -> {len(files_to_process)}/{total_files} (샘플) 변환 진행")

    converted_count = 0
    for json_path in files_to_process:
        base_filename = os.path.basename(json_path)
        img_filename = os.path.splitext(base_filename)[0] + ".jpg"
        img_path = os.path.join(img_src_dir, img_filename)

        if not os.path.exists(img_path):
            print(f"이미지 누락: {img_path}")
            continue

        try:
            with Image.open(img_path) as img:
                img_width, img_height = img.size
        except Exception as exc:
            print(f"이미지 열기 실패: {img_path} ({exc})")
            continue

        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        annotations = data.get("annotations", [])
        if not annotations:
            continue

        yolo_lines = []
        for anno in annotations:
            category_id = anno.get("category_id")
            class_id = resolve_class_id(category_id)
            if class_id is None:
                continue

            bbox = anno.get("bbox", [])
            if len(bbox) != 4:
                continue

            x_center, y_center, width, height = normalize_coco_bbox(img_width, img_height, bbox)
            yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            yolo_lines.append(yolo_line)

        if not yolo_lines:
            continue

        shutil.copy2(img_path, os.path.join(img_dest_dir, img_filename))

        label_dest_path = os.path.join(lbl_dest_dir, os.path.splitext(base_filename)[0] + ".txt")
        with open(label_dest_path, "w", encoding="utf-8") as out_f:
            out_f.write("\n".join(yolo_lines))
        converted_count += 1

    print(f"{lbl_src_dir}: 변환 완료 ({converted_count}개)")


# ------------------------------------------------------------
# 단계 4: 데이터 변환 실행
# ------------------------------------------------------------

print("\n단계 4: 데이터 변환 실행")

convert_data(train_good_img_path, train_good_lbl_path,
             os.path.join(dataset_base_path, "images/train"),
             os.path.join(dataset_base_path, "labels/train"))

convert_data(train_bad_img_path, train_bad_lbl_path,
             os.path.join(dataset_base_path, "images/train"),
             os.path.join(dataset_base_path, "labels/train"))

convert_data(val_good_img_path, val_good_lbl_path,
             os.path.join(dataset_base_path, "images/val"),
             os.path.join(dataset_base_path, "labels/val"))

convert_data(val_bad_img_path, val_bad_lbl_path,
             os.path.join(dataset_base_path, "images/val"),
             os.path.join(dataset_base_path, "labels/val"))

convert_data(train_blue_hole_img_path, train_blue_hole_lbl_path,
             os.path.join(dataset_base_path, "images/train"),
             os.path.join(dataset_base_path, "labels/train"),
             skip_sampling=True)

convert_data(val_blue_hole_img_path, val_blue_hole_lbl_path,
             os.path.join(dataset_base_path, "images/val"),
             os.path.join(dataset_base_path, "labels/val"),
             skip_sampling=True)

print("단계 4: 데이터 변환 완료")


# ------------------------------------------------------------
# 단계 5: 데이터셋 YAML 작성
# ------------------------------------------------------------

print("\n단계 5: data.yaml 작성")

yaml_content = f"""
train: {os.path.join(dataset_base_path, 'images/train')}
val: {os.path.join(dataset_base_path, 'images/val')}

nc: {len(CLASS_ID_MAP)}
names: ['good_weld', 'bad_weld', 'bluehole_weld']
"""

yaml_path = os.path.join(dataset_base_path, "weld_multiclass.yaml")
with open(yaml_path, "w", encoding="utf-8") as yaml_file:
    yaml_file.write(yaml_content)

print(f"{yaml_path} 작성 완료")
print("--- YAML 미리보기 ---")
print(yaml_content)


# ------------------------------------------------------------
# 단계 6: YOLO 모델 학습 및 최종 모델 저장
# ------------------------------------------------------------

print("\n단계 6: 모델 학습 시작")

import os, shutil

model = YOLO("yolov8n.pt")

# Colab 환경에서는 __file__이 없기 때문에 현재 작업 디렉토리를 기준으로 설정

target_project = base_drive_path / "weld_inspection_multiclass"
target_run_name = "run_10_percent"
application_model_filename = "welding_multiclass.pt"


results = None
try:
    results = model.train(
        data=yaml_path,
        epochs=10,
        imgsz=640,
        project=str(target_project),   # 절대경로로 지정
        name=target_run_name,
        save_period=1,                 # ✅ 각 Epoch마다 체크포인트 저장
    )
    print("단계 6: 모델 학습 완료")
except Exception as e:
    print(f"모델 학습 중 오류: {e}")

# ------------------------------------------------------------
# 최종 모델 복사 (best.pt 또는 last.pt → 현재 작업 디렉토리)
# ------------------------------------------------------------

weights_dir = target_project / target_run_name / "weights"
best_path = weights_dir / "best.pt"
last_path = weights_dir / "last.pt"
final_model_path = base_drive_path / application_model_filename

if best_path.exists():
    shutil.copy2(best_path, final_model_path)
    print(f"최종 모델 복사 완료 (best.pt): {final_model_path}")
elif last_path.exists():
    shutil.copy2(last_path, final_model_path)
    print(f"최종 모델 복사 완료 (last.pt): {final_model_path}")
else:
    print(f"경고: {weights_dir} 안에서 best.pt / last.pt를 찾지 못했습니다.")



# ------------------------------------------------------------
# 단계 7: 검증 및 간단 추론 (선택)
# ------------------------------------------------------------

print("\n단계 7: 검증 및 추론 예시")

trained_model_path = f"{target_project}/{target_run_name}/weights/best.pt"

if os.path.exists(trained_model_path):
    print(f"학습된 모델 로드: {trained_model_path}")

    print("\n검증 수행 중...")
    !yolo task=detect mode=val model={trained_model_path} data={yaml_path}

    try:
        sample_image = glob.glob(os.path.join(dataset_base_path, "images/val", "*.jpg"))[0]
        print(f"\n추론 수행 (샘플 이미지: {sample_image})")
        !yolo task=detect mode=predict model={trained_model_path} source={sample_image} save=True
    except IndexError:
        print("추론에 사용할 검증 이미지가 없습니다.")
else:
    print(f"경고: {trained_model_path} 가 존재하지 않습니다. 학습 결과를 확인하세요.")

print("\n프로세스 종료")



단계 1: 환경 설정 시작
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive 마운트 성공
Ultralytics YOLO 설치 진행
단계 1: 환경 설정 완료

단계 2: 데이터 경로 및 카테고리 정의
단계 2: 카테고리 및 경로 설정 완료

단계 4: 데이터 변환 실행
/content/drive/MyDrive/Hackathon/Training/label/TL_용접_용접양품_조인트 -> 549/5497 (샘플) 변환 진행
/content/drive/MyDrive/Hackathon/Training/label/TL_용접_용접양품_조인트: 변환 완료 (549개)
/content/drive/MyDrive/Hackathon/Training/label/TL_용접_용접불량_조인트 -> 538/5383 (샘플) 변환 진행
/content/drive/MyDrive/Hackathon/Training/label/TL_용접_용접불량_조인트: 변환 완료 (538개)
/content/drive/MyDrive/Hackathon/Validation/label/VL_용접_용접양품_조인트 -> 70/700 (샘플) 변환 진행
/content/drive/MyDrive/Hackathon/Validation/label/VL_용접_용접양품_조인트: 변환 완료 (70개)
/content/drive/MyDrive/Hackathon/Validation/label/VL_용접_용접불량_조인트 -> 66/668 (샘플) 변환 진행
/content/drive/MyDrive/Hackathon/Validation/label/VL_용접_용접불량_조인트: 변환 완료 (66개)
/content/drive/MyDrive/Hackathon/Training/label/TL_용접_용접블루홀_조인트 -> 150/150 (샘플) 변

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
